<a href="https://colab.research.google.com/github/oijiin/ANALISE_DE_DADOS/blob/main/SIMULADOR_integracao_WMS_TMS_ERP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Proposta de Código Inicial (Simulador WMS Básico em Python)**

Vamos criar uma estrutura inicial usando Programação Orientada a Objetos (OOP), que se encaixa muito bem na modelagem de entidades do mundo real como itens, locais de estoque e o próprio armazém.

In [1]:
import json
import datetime
import time
import uuid
from enum import Enum, auto
from collections import namedtuple, defaultdict
import heapq
import os # Para verificar se arquivo existe

In [2]:
# Exceções Customizadas

class WMSError(Exception):
    """
      Classe base para exceções do WMS.
    """
    def __init__(self, message):
        super().__init__(message)
        self.message = message
        self.timestamp = datetime.datetime.now()
        self.type = self.__class__.__name__
        self.traceback = None
        self.stacktrace = None

    def __str__(self):
        return f"{self.type} - {self.message} - Timestamp: {self.timestamp}"

class SKUInvalidoError(WMSError):
    """Exceção para quando um SKU é inválido."""
    def __init__(self, sku):
        super().__init__(f"SKU inválido: {sku}")
        self.sku = sku

    pass

class ItemNaoEncontradoError(WMSError):
    """Exceção para quando um SKU não está no catálogo."""
    def __init__(self, sku):
        super().__init__(f"Item com SKU {sku} não encontrado no catálogo.")
        self.sku = sku

class QuantidadeInvalidaError(WMSError):
    """Exceção para quando a quantidade é inválida."""
    def __init__(self, quantidade):
        super().__init__(f"Quantidade inválida: {quantidade}")
        self.quantidade = quantidade

class PedidoInvalidoError(WMSError):
    """Exceção para quando um pedido é inválido."""
    def __init__(self, pedido):
        super().__init__(f"Pedido inválido: {pedido}")
        self.pedido = pedido

class PedidoJaExisteError(WMSError):
    """Exceção para quando um pedido já existe."""
    def __init__(self, pedido_id):
        super().__init__(f"Pedido com ID {pedido_id} já existe.")

class PedidoNaoEncontradoError(WMSError):
    """Exceção para quando um pedido não é encontrado."""
    def __init__(self, pedido_id):
        super().__init__(f"Pedido com ID {pedido_id} não encontrado.")
        self.pedido_id = pedido_id

class LocalizacaoNaoEncontradaError(WMSError):
    """Exceção para quando um ID de localização não existe."""
    def __init__(self, id_local):
        super().__init__(f"Localização com ID {id_local} não encontrada.")
        self.id_local = id_local

class EstoqueInsuficienteError(WMSError):
    """Exceção para falta de estoque."""
    def __init__(self, sku, id_local, disponivel, solicitado):
        super().__init__(f"Estoque insuficiente do SKU {sku} em {id_local}. Disponível: {disponivel}, Solicitado: {solicitado}.")
        self.sku = sku
        self.id_local = id_local
        self.disponivel = disponivel
        self.solicitado = solicitado

class CapacidadeExcedidaError(WMSError):
    """Exceção para quando a capacidade da localização é excedida."""
    def __init__(self, id_local, tipo_capacidade, maximo, atual, tentando_adicionar):
        super().__init__(f"Capacidade '{tipo_capacidade}' ({maximo}) da localização {id_local} excedida. Ocupação atual: {atual}, Tentando adicionar: {tentando_adicionar}.")
        self.id_local = id_local
        self.tipo_capacidade = tipo_capacidade
        self.maximo = maximo
        self.atual = atual
        self.tentando_adicionar = tentando_adicionar

class NenhumaLocalizacaoDisponivelError(WMSError):
    """Exceção para quando não há local adequado para guardar/pegar um item."""
    def __init__(self, sku, quantidade, operacao="armazenar"):
        super().__init__(f"Nenhuma localização adequada encontrada para {operacao} {quantidade} unidade(s) do SKU {sku}.")
        self.sku = sku
        self.quantidade = quantidade
        self.operacao = operacao

In [3]:
# --- Enum para Status do Pedido ---
class StatusPedido(Enum):
    PENDENTE = auto()
    EM_PICKING = auto()
    PICKING_FALHOU = auto()
    PICKING_COMPLETO = auto()
    EXPEDIDO = auto()
    CANCELADO = auto()

    def __str__(self):
        return self.name # Representação em string amigável

In [4]:
# --- Estrutura de Dados para Lotes ---

# Usaremos namedtuple para clareza ao acessar os dados do lote
# Adiciona to_dict/from_dict para serialização
LoteBase = namedtuple("LoteBase", ["quantidade", "timestamp_entrada", "id_lote"])
class Lote(LoteBase):
    def to_dict(self) -> dict:
        return {
            'quantidade': self.quantidade,
            'timestamp_entrada': self.timestamp_entrada.isoformat(),
            'id_lote': self.id_lote
        }

    @classmethod
    def from_dict(cls, data: dict) -> 'Lote':
        return cls(
            quantidade=data['quantidade'],
            timestamp_entrada=datetime.datetime.fromisoformat(data['timestamp_entrada']),
            id_lote=data['id_lote']
        )

In [5]:
# --- Classe Item (com to_dict) ---
class Item:
    """Representa tipo de item (SKU) com atributos físicos."""
    def __init__(self, sku, nome, descricao="", volume=0.0, peso=0.0):
        # Validações omitidas para brevidade (são as mesmas de antes)
        if not sku or not isinstance(sku, str): raise ValueError("SKU inválido.")
        if not nome or not isinstance(nome, str): raise ValueError("Nome inválido.")
        if not isinstance(volume, (int, float)) or volume < 0: raise ValueError("Volume inválido.")
        if not isinstance(peso, (int, float)) or peso < 0: raise ValueError("Peso inválido.")
        self.sku = sku
        self.nome = nome
        self.descricao = descricao
        self.volume = float(volume)
        self.peso = float(peso)

    def to_dict(self) -> dict:
        """Converte para dicionário serializável."""
        return {
            'sku': self.sku, 'nome': self.nome, 'descricao': self.descricao,
            'volume': self.volume, 'peso': self.peso
        }

    # Não precisamos de from_dict aqui, pois __init__ recebe os mesmos args
    def __str__(self):
        return f"Item(SKU: {self.sku}, Nome: {self.nome}, Vol: {self.volume:.2f}, Peso: {self.peso:.2f})"

    def __repr__(self):
        return (f"Item(sku='{self.sku}', nome='{self.nome}', descricao='{self.descricao}', "
                f"volume={self.volume}, peso={self.peso})")

In [6]:
# --- Classe Localizacao (com to_dict/from_dict e remoção do hack) ---
class Localizacao:
    """Representa localização com capacidade e estoque por lotes."""
    def __init__(self, id_local, capacidade_maxima=float('inf'), tipo_capacidade='volume'):
        # Validações omitidas para brevidade
        if not id_local or not isinstance(id_local, str): raise ValueError("ID Local inválido.")
        if tipo_capacidade not in ['volume', 'peso', 'quantidade']: raise ValueError("Tipo capacidade inválido.")
        # Validação de capacidade omitida

        self.id_local = id_local
        self.inventario: defaultdict[str, list[Lote]] = defaultdict(list)
        self.capacidade_maxima = float(capacidade_maxima)
        self.tipo_capacidade = tipo_capacidade
        self._ocupacao_cache = {'volume': 0.0, 'peso': 0.0, 'quantidade': 0}
        self._cache_calculado = False # Flag para saber se o cache foi populado

    # Métodos _calcular_atributo_lote, calcular_ocupacao_atual, verificar_capacidade
    # adicionar_lote, remover_item, verificar_estoque (permanecem os mesmos de antes)
    # apenas garantir que usem `Item` e `Lote` corretamente

    def _calcular_atributo_lote(self, lote: Lote, sku_item: Item, atributo: str):
        """Calcula o volume, peso ou quantidade de um lote."""
        if atributo == 'quantidade':
            return lote.quantidade
        elif atributo == 'volume':
            return lote.quantidade * sku_item.volume
        elif atributo == 'peso':
            return lote.quantidade * sku_item.peso
        return 0.0

    def calcular_ocupacao_atual(self, catalogo_itens: dict[str, Item], recalcular=False):
        """Calcula e retorna a ocupação atual baseada no tipo de capacidade."""
        if not recalcular and self._cache_calculado:
            return self._ocupacao_cache[self.tipo_capacidade]

        total_volume = 0.0
        total_peso = 0.0
        total_quantidade = 0

        for sku, lotes in self.inventario.items():
            if sku not in catalogo_itens:
                 print(f"AVISO: SKU {sku} encontrado em {self.id_local} mas não no catálogo durante cálculo de ocupação.")
                 continue
            item_def = catalogo_itens[sku]
            for lote in lotes:
                total_volume += lote.quantidade * item_def.volume
                total_peso += lote.quantidade * item_def.peso
                total_quantidade += lote.quantidade

        self._ocupacao_cache['volume'] = total_volume
        self._ocupacao_cache['peso'] = total_peso
        self._ocupacao_cache['quantidade'] = total_quantidade
        self._cache_calculado = True # Marca que o cache foi calculado

        return self._ocupacao_cache[self.tipo_capacidade]

    def verificar_capacidade(self, sku_para_adicionar: Item, quantidade: int, catalogo_itens: dict) -> bool:
        """Verifica se adicionar o item excederia a capacidade."""
        if self.capacidade_maxima == float('inf'):
            return True

        ocupacao_atual = self.calcular_ocupacao_atual(catalogo_itens) # Garante que está calculado
        valor_a_adicionar = 0.0

        if self.tipo_capacidade == 'quantidade':
            valor_a_adicionar = quantidade
        elif self.tipo_capacidade == 'volume':
            valor_a_adicionar = quantidade * sku_para_adicionar.volume
        elif self.tipo_capacidade == 'peso':
            valor_a_adicionar = quantidade * sku_para_adicionar.peso

        # Usar uma pequena tolerância para comparações de float
        tolerancia = 1e-9
        if ocupacao_atual + valor_a_adicionar > self.capacidade_maxima + tolerancia:
            raise CapacidadeExcedidaError(
                self.id_local, self.tipo_capacidade, self.capacidade_maxima,
                ocupacao_atual, valor_a_adicionar
            )
        return True

    def adicionar_lote(self, sku_item: Item, quantidade: int, timestamp_entrada: datetime.datetime, catalogo_itens: dict):
        """Adiciona um novo lote, verificando capacidade e atualizando cache."""
        if not isinstance(quantidade, int) or quantidade <= 0:
            raise ValueError("Quantidade inválida.")

        self.verificar_capacidade(sku_item, quantidade, catalogo_itens) # Pode levantar CapacidadeExcedidaError

        id_lote = timestamp_entrada.isoformat() + f"_{sku_item.sku}" # ID único por timestamp+sku
        novo_lote = Lote(quantidade=quantidade, timestamp_entrada=timestamp_entrada, id_lote=id_lote)
        self.inventario[sku_item.sku].append(novo_lote)

        # Atualizar ocupação (incrementalmente)
        self.calcular_ocupacao_atual(catalogo_itens, recalcular=True) # Força recalculo ao adicionar

        # Log é feito pelo ArmazemWMS
        # print(f"INFO: Lote {id_lote} ({quantidade}x SKU {sku_item.sku}) adicionado à localização {self.id_local}.")

    def remover_item(self, sku: str, quantidade_solicitada: int, catalogo_itens: dict, strategy='FIFO') -> list[Lote]:
        """Remove itens, atualiza cache e retorna lotes afetados."""
        if not isinstance(quantidade_solicitada, int) or quantidade_solicitada <= 0:
            raise ValueError("Quantidade solicitada inválida.")
        if sku not in self.inventario or not self.inventario[sku]:
            raise EstoqueInsuficienteError(sku, self.id_local, 0, quantidade_solicitada)

        estoque_total_sku = self.verificar_estoque(sku)
        if estoque_total_sku < quantidade_solicitada:
             raise EstoqueInsuficienteError(sku, self.id_local, estoque_total_sku, quantidade_solicitada)

        lotes = self.inventario[sku]
        quantidade_restante = quantidade_solicitada
        lotes_removidos_info = []

        if strategy == 'FIFO':
            lotes.sort(key=lambda lote: lote.timestamp_entrada)
        elif strategy == 'LIFO':
            lotes.sort(key=lambda lote: lote.timestamp_entrada, reverse=True)
        else:
            raise ValueError("Estratégia de remoção inválida.")

        indices_para_remover = []
        lotes_para_manter = []

        for i, lote in enumerate(lotes):
            if quantidade_restante <= 0:
                lotes_para_manter.append(lote)
                continue

            quantidade_a_remover_deste_lote = min(lote.quantidade, quantidade_restante)
            # Cria info do lote removido com a quantidade efetiva
            lotes_removidos_info.append(Lote(quantidade_a_remover_deste_lote, lote.timestamp_entrada, lote.id_lote))
            quantidade_restante -= quantidade_a_remover_deste_lote

            if lote.quantidade > quantidade_a_remover_deste_lote:
                lote_atualizado = Lote(lote.quantidade - quantidade_a_remover_deste_lote,
                                       lote.timestamp_entrada,
                                       lote.id_lote)
                lotes_para_manter.append(lote_atualizado)

        if not lotes_para_manter:
            del self.inventario[sku]
        else:
            self.inventario[sku] = lotes_para_manter

        # Atualizar a ocupação
        self.calcular_ocupacao_atual(catalogo_itens, recalcular=True)

        # print(f"INFO: Removido {quantidade_solicitada}x SKU {sku} de {self.id_local} (Estratégia: {strategy}). Lotes afetados: {lotes_removidos_info}")
        return lotes_removidos_info

    def verificar_estoque(self, sku_item):
        """Retorna a quantidade total de um item."""
        return sum(lote.quantidade for lote in self.inventario.get(sku_item, []))

    def to_dict(self) -> dict:
        """Converte para dicionário serializável."""
        # Converte o inventário (defaultdict) para um dict normal com lotes serializados
        inventario_serializado = {
            sku: [lote.to_dict() for lote in lotes]
            for sku, lotes in self.inventario.items()
        }
        return {
            'id_local': self.id_local,
            'capacidade_maxima': self.capacidade_maxima if self.capacidade_maxima != float('inf') else 'inf', # Salva 'inf' como string
            'tipo_capacidade': self.tipo_capacidade,
            'inventario': inventario_serializado
            # Não salvamos o cache de ocupação, será recalculado ao carregar se necessário
        }

    @classmethod
    def from_dict(cls, data: dict, catalogo_global: dict[str, Item]) -> 'Localizacao':
        """Cria instância a partir de dicionário."""
        capacidade = data['capacidade_maxima']
        if capacidade == 'inf':
            capacidade = float('inf')

        loc = cls(
            id_local=data['id_local'],
            capacidade_maxima=float(capacidade),
            tipo_capacidade=data['tipo_capacidade']
        )
        # Recria o inventário a partir dos lotes serializados
        loc.inventario = defaultdict(list)
        for sku, lotes_data in data.get('inventario', {}).items():
             if sku in catalogo_global: # Garante que o SKU existe no catalogo atual
                loc.inventario[sku] = [Lote.from_dict(lote_data) for lote_data in lotes_data]
             else:
                print(f"AVISO ao carregar {loc.id_local}: SKU {sku} do inventario salvo não existe mais no catalogo. Ignorando estoque.")
        # Cache será recalculado quando necessário (ex: na primeira chamada a calcular_ocupacao_atual)
        return loc

    def __str__(self):
        # Removido o cálculo de ocupação do __str__ para evitar dependência do catálogo aqui
        itens_str_list = []
        for sku, lotes in sorted(self.inventario.items()): # Ordena SKUs para consistência
            qtd_total = sum(l.quantidade for l in lotes)
            itens_str_list.append(f"{sku}: {qtd_total} ({len(lotes)} Lotes)")

        itens_str = ", ".join(itens_str_list) if itens_str_list else "Vazia"
        cap_str = f"{self.capacidade_maxima:.2f}" if self.capacidade_maxima != float('inf') else "inf"
        return (f"Localizacao(ID: {self.id_local}, "
                f"Cap: {cap_str} {self.tipo_capacidade}, "
                # f"Ocup: ???, " # Removido para simplificar __str__
                f"Inventario: [{itens_str}])")

In [7]:
# --- Classe Pedido ---
class Pedido:
    """Representa um pedido de cliente."""
    def __init__(self, cliente: str, itens: dict[str, int], pedido_id: str | None = None, status: StatusPedido = StatusPedido.PENDENTE):
        if not cliente or not isinstance(cliente, str):
            raise ValueError("Nome do cliente inválido.")
        if not isinstance(itens, dict) or not all(isinstance(k, str) and isinstance(v, int) and v > 0 for k, v in itens.items()):
            raise ValueError("Itens do pedido inválidos. Deve ser um dicionário {sku: quantidade_positiva}.")
        if status not in StatusPedido:
            raise ValueError("Status inicial inválido.")

        self.pedido_id = pedido_id or str(uuid.uuid4()) # Gera ID único se não for fornecido
        self.cliente = cliente
        # Guarda uma cópia para evitar modificações externas inesperadas
        self.itens_solicitados = dict(itens)
        self.status = status
        self.timestamp_criacao = datetime.datetime.now()
        # Poderíamos adicionar campos para rastrear o que foi efetivamente separado/expedido

    def mudar_status(self, novo_status: StatusPedido):
        if novo_status not in StatusPedido:
             raise ValueError(f"Status inválido: {novo_status}")
        print(f"INFO Pedido {self.pedido_id}: Mudando status de {self.status} para {novo_status}")
        self.status = novo_status
        # Aqui poderíamos adicionar lógica, como registrar timestamp da mudança de status

    def to_dict(self) -> dict:
        """Converte o objeto Pedido para um dicionário serializável em JSON."""
        return {
            'pedido_id': self.pedido_id,
            'cliente': self.cliente,
            'itens_solicitados': self.itens_solicitados,
            'status': self.status.name, # Salva o nome do enum
            'timestamp_criacao': self.timestamp_criacao.isoformat()
        }

    @classmethod
    def from_dict(cls, data: dict) -> 'Pedido':
        """Cria um objeto Pedido a partir de um dicionário."""
        pedido = cls(
            cliente=data['cliente'],
            itens=data['itens_solicitados'], # Garante que o __init__ valide
            pedido_id=data['pedido_id'],
            status=StatusPedido[data['status']] # Converte string de volta para enum
        )
        # Sobrescreve o timestamp com o valor carregado
        pedido.timestamp_criacao = datetime.datetime.fromisoformat(data['timestamp_criacao'])
        return pedido

    def __str__(self):
        itens_str = ", ".join(f"{sku}:{qty}" for sku, qty in self.itens_solicitados.items())
        return (f"Pedido(ID: {self.pedido_id}, Cliente: {self.cliente}, Status: {self.status}, "
                f"Itens: [{itens_str}], Criado: {self.timestamp_criacao.strftime('%Y-%m-%d %H:%M')})")

    def __repr__(self):
        return (f"Pedido(pedido_id='{self.pedido_id}', cliente='{self.cliente}', "
                f"itens={self.itens_solicitados}, status=<{self.status}>)")

In [8]:
# --- Classe ArmazemWMS (com persistência e gerenciamento de pedidos) ---
class ArmazemWMS:
    """Orquestra WMS com persistência e gerenciamento de pedidos."""

    def __init__(self, nome):
        self.nome = nome
        self.localizacoes: dict[str, Localizacao] = {}
        self.catalogo_itens: dict[str, Item] = {}
        self.log_movimentacoes: list[dict] = []
        self.pedidos: dict[str, Pedido] = {} # Dicionário para armazenar pedidos

    def adicionar_item_catalogo(self, item: Item): # Mesmo de antes
        if not isinstance(item, Item): raise TypeError("Objeto não é Item.")
        if item.sku in self.catalogo_itens: print(f"AVISO: SKU {item.sku} já existe. Substituindo.")
        self.catalogo_itens[item.sku] = item
        print(f"INFO: Item {item.sku} ({item.nome}) adicionado/atualizado no catálogo.")

    def adicionar_localizacao(self, localizacao: Localizacao): # Mesmo de antes
        if not isinstance(localizacao, Localizacao): raise TypeError("Objeto não é Localizacao.")
        if localizacao.id_local in self.localizacoes: print(f"AVISO: Localização {localizacao.id_local} já existe."); return False
        self.localizacoes[localizacao.id_local] = localizacao
        print(f"INFO: Localização {localizacao.id_local} adicionada.")
        return True

    def get_item(self, sku: str) -> Item: # Mesmo de antes
        item = self.catalogo_itens.get(sku)
        if not item: raise ItemNaoEncontradoError(sku)
        return item

    def get_localizacao(self, id_local: str) -> Localizacao: # Mesmo de antes
        loc = self.localizacoes.get(id_local)
        if not loc: raise LocalizacaoNaoEncontradaError(id_local)
        return loc

    def adicionar_pedido(self, pedido: Pedido):
        """Adiciona um novo pedido ao WMS."""
        if not isinstance(pedido, Pedido):
            raise TypeError("Objeto fornecido não é do tipo Pedido.")
        if pedido.pedido_id in self.pedidos:
            print(f"AVISO: Pedido com ID {pedido.pedido_id} já existe.")
            return False
        # Verifica se todos os itens do pedido existem no catálogo
        for sku in pedido.itens_solicitados:
             self.get_item(sku) # Levanta ItemNaoEncontradoError se não existir

        self.pedidos[pedido.pedido_id] = pedido
        print(f"INFO: Pedido {pedido.pedido_id} (Cliente: {pedido.cliente}) adicionado.")
        return True

    def buscar_pedido(self, pedido_id: str) -> Pedido:
        """Busca um pedido pelo ID."""
        pedido = self.pedidos.get(pedido_id)
        if not pedido:
            raise PedidoNaoEncontradoError(f"Pedido com ID {pedido_id} não encontrado.")
        return pedido

    def registrar_movimentacao(self, tipo, sku, quantidade, id_local_origem=None, id_local_destino=None, detalhes="", lotes_afetados=None, pedido_id=None):
        """Registra um evento no log, opcionalmente associado a um pedido."""
        timestamp = datetime.datetime.now().isoformat()
        log_entry = {
            "timestamp": timestamp, "tipo": tipo, "sku": sku, "quantidade": quantidade,
            "origem": id_local_origem, "destino": id_local_destino, "detalhes": detalhes,
            "lotes": [l.to_dict() for l in lotes_afetados] if lotes_afetados else [], # Serializa Lotes aqui
            "pedido_id": pedido_id # Novo campo
        }
        self.log_movimentacoes.append(log_entry)

    # --- Estratégias de Armazenagem/Picking (_encontrar_local_para_guardar, _encontrar_lotes_para_picking)
    # Permanecem as mesmas de antes. Garantir que usam `get_item`, `get_localizacao`.

    def _encontrar_local_para_guardar(self, sku: str, quantidade: int, item_def: Item, estrategia='EXISTENTE_SKU') -> str:
        # Código igual ao anterior (verifica capacidade, aplica estratégia)
        candidatos = []
        locs_a_ignorar = {"RECEBIMENTO", "PICKING_AREA", "EXPEDICAO"} # Configurável talvez?
        for id_loc, loc in self.localizacoes.items():
             if id_loc in locs_a_ignorar: continue
             try:
                 loc.verificar_capacidade(item_def, quantidade, self.catalogo_itens)
                 # Usa o cache se disponível, recalcula se não
                 ocupacao = loc.calcular_ocupacao_atual(self.catalogo_itens)
                 ocupacao_relativa = ocupacao / loc.capacidade_maxima if loc.capacidade_maxima != float('inf') else 0
                 ja_contem_sku = sku in loc.inventario and loc.inventario[sku]
                 candidatos.append({"id": id_loc, "ocupacao_relativa": ocupacao_relativa, "ja_contem_sku": ja_contem_sku})
             except CapacidadeExcedidaError: continue

        if not candidatos: raise NenhumaLocalizacaoDisponivelError(sku, quantidade, "armazenar")

        # Aplica Estratégia (igual ao anterior)
        if estrategia == 'PRIMEIRO_DISPONIVEL': return candidatos[0]['id']
        elif estrategia == 'MENOR_OCUPACAO': return min(candidatos, key=lambda c: c['ocupacao_relativa'])['id']
        elif estrategia == 'EXISTENTE_SKU':
             existentes = [c for c in candidatos if c['ja_contem_sku']]
             if existentes: return min(existentes, key=lambda c: c['ocupacao_relativa'])['id']
             else: return min(candidatos, key=lambda c: c['ocupacao_relativa'])['id'] # Fallback
        else: # Default: Melhor fit (EXISTENTE_SKU)
             existentes = [c for c in candidatos if c['ja_contem_sku']]
             if existentes: return min(existentes, key=lambda c: c['ocupacao_relativa'])['id']
             else: return min(candidatos, key=lambda c: c['ocupacao_relativa'])['id']

    def _encontrar_lotes_para_picking(self, sku: str, quantidade_solicitada: int, strategy='FIFO') -> list[tuple[str, list[Lote]]]:
        # Código igual ao anterior (busca global, ordena por timestamp, seleciona lotes)
        if strategy not in ['FIFO', 'LIFO']: raise ValueError("Estratégia de picking inválida.")

        candidatos_lotes = [] # (timestamp, quantidade, id_lote, id_local)
        locs_a_ignorar = {"RECEBIMENTO", "PICKING_AREA", "EXPEDICAO"}
        for id_loc, loc in self.localizacoes.items():
             if id_loc in locs_a_ignorar: continue
             if sku in loc.inventario:
                 for lote in loc.inventario[sku]:
                     candidatos_lotes.append((lote.timestamp_entrada, lote.quantidade, lote.id_lote, id_loc))

        total_disponivel_armazenagem = sum(l[1] for l in candidatos_lotes)
        if total_disponivel_armazenagem < quantidade_solicitada:
            raise EstoqueInsuficienteError(sku, "ARMAZÉM (Armazenagem)", total_disponivel_armazenagem, quantidade_solicitada)

        if strategy == 'FIFO': candidatos_lotes.sort(key=lambda x: x[0])
        else: candidatos_lotes.sort(key=lambda x: x[0], reverse=True)

        quantidade_a_pegar = quantidade_solicitada
        picking_plan = defaultdict(list) # {id_local: [lotes_a_pegar_neste_local]}

        for ts, qtd_lote, id_lote, id_loc in candidatos_lotes:
            if quantidade_a_pegar <= 0: break
            qtd_deste_lote = min(qtd_lote, quantidade_a_pegar)
            lote_info = Lote(qtd_deste_lote, ts, id_lote) # Lote com qtd a pegar
            picking_plan[id_loc].append(lote_info)
            quantidade_a_pegar -= qtd_deste_lote

        if quantidade_a_pegar > 0: # Checagem dupla, deve ser pega pela verificação inicial
            raise EstoqueInsuficienteError(sku, "ARMAZÉM (Seleção Pós-Sort)", total_disponivel_armazenagem, quantidade_solicitada)

        return list(picking_plan.items())

    # --- Operações WMS Atualizadas ---

    def receber_mercadoria(self, sku: str, quantidade: int, id_local_recebimento="RECEBIMENTO"): # Mesmo de antes
        try:
            item_def = self.get_item(sku)
            loc_recebimento = self.get_localizacao(id_local_recebimento)
            timestamp = datetime.datetime.now()
            # Adicionar lote lida com capacidade
            loc_recebimento.adicionar_lote(item_def, quantidade, timestamp, self.catalogo_itens)
            self.registrar_movimentacao("RECEBIMENTO", sku, quantidade,
                                       id_local_destino=id_local_recebimento, detalhes="Mercadoria recebida.",
                                       lotes_afetados=[Lote(quantidade, timestamp, timestamp.isoformat()+f"_{sku}")])
        except (ItemNaoEncontradoError, LocalizacaoNaoEncontradaError, CapacidadeExcedidaError, ValueError) as e:
            print(f"ERRO [Recebimento]: {e}")

    def guardar_mercadoria(self, sku: str, quantidade: int, id_local_origem: str, estrategia_guarda='EXISTENTE_SKU'): # Mesmo de antes
        # Lógica com rollback, usa _encontrar_local_para_guardar
        try:
            item_def = self.get_item(sku)
            loc_origem = self.get_localizacao(id_local_origem)

            # Encontrar destino ANTES de remover da origem
            # Nota: Usamos a quantidade total para verificar capacidade, embora possa vir de múltiplos lotes.
            # Idealmente, verificaríamos lote a lote se a capacidade fosse muito restrita.
            id_local_destino = self._encontrar_local_para_guardar(sku, quantidade, item_def, estrategia_guarda)
            loc_destino = self.get_localizacao(id_local_destino)

            # Remover da origem (usando FIFO como padrão para transferências internas)
            # Importante: O método remover_item retorna os detalhes dos lotes removidos.
            # Change is here:
            lotes_removidos = loc_origem.remover_item(sku, quantidade, self.catalogo_itens, strategy='FIFO')

            # Adicionar ao destino, criando NOVOS lotes com o timestamp ATUAL
            # (Perdemos o rastreio do timestamp original na transferência nesta implementação)
            timestamp_movimentacao = datetime.datetime.now()
            lotes_adicionados_destino = []
            try:
                for lote_removido in lotes_removidos_origem:
                    # Usar um ID único para o novo lote no destino
                    novo_id_lote = timestamp_mov.isoformat() + f"_{sku}_M" # Adiciona _M para indicar movimentado
                    loc_destino.adicionar_lote(item_def, lote_removido.quantidade, timestamp_mov, self.catalogo_itens)
                    lotes_adicionados_destino.append(Lote(lote_removido.quantidade, timestamp_mov, novo_id_lote))

                self.registrar_movimentacao("ARMAZENAGEM", sku, quantidade,
                                           id_local_origem=id_local_origem, id_local_destino=id_local_destino,
                                           detalhes=f"Movido para estoque via estratégia {estrategia_guarda}.",
                                           lotes_afetados=lotes_adicionados_destino,
                                           pedido_id=None) # Guarda não está ligado a pedido

            except Exception as erro_adicao:
                print(f"ERRO CRÍTICO [Guardar - Adição Destino]: {erro_adicao}. Iniciando rollback.")
                # Rollback: Devolver lotes à origem
                try:
                    ts_rollback = datetime.datetime.now()
                    for lote_dev in lotes_removidos_origem: # Devolve o que foi removido
                        # Idealmente, o lote devolvido deveria ter um timestamp que indique o rollback
                        # Usando o timestamp original da remoção + indicativo
                        id_lote_rollback = lote_dev.id_lote + "_ROLLBACK"
                        loc_origem.adicionar_lote(item_def, lote_dev.quantidade, lote_dev.timestamp_entrada, self.catalogo_itens)
                        # Log de rollback
                    print(f"INFO [Guardar - Rollback]: {quantidade}x {sku} devolvido(s) para {id_local_origem}.")
                    self.registrar_movimentacao("ROLLBACK_ARMAZENAGEM", sku, quantidade,
                                               id_local_origem=id_local_destino, id_local_destino=id_local_origem,
                                               detalhes=f"Falha ao adicionar em {id_local_destino}, devolvido para origem.",
                                               lotes_afetados=lotes_removidos_origem) # Lotes originais afetados
                except Exception as erro_rollback:
                    print(f"ERRO CRÍTICO IMPOSSÍVEL DE REVERTER [Guardar - Rollback]: {erro_rollback}. Estado inconsistente!")
                    self.registrar_movimentacao("ERRO_CRITICO_ROLLBACK", sku, quantidade,
                                               id_local_origem=id_local_destino, id_local_destino=id_local_origem,
                                               detalhes=f"Falha CRITICA ao reverter armazenagem! INCONSISTENTE.")

        except (ItemNaoEncontradoError, LocalizacaoNaoEncontradaError, EstoqueInsuficienteError,
                CapacidadeExcedidaError, NenhumaLocalizacaoDisponivelError, ValueError) as e:
            print(f"ERRO [Guardar Mercadoria]: {e}")

    def _executar_picking_item(self, sku: str, quantidade: int, estrategia_picking: str, id_local_picking: str, pedido_id: str | None) -> bool:
        """Lógica interna para fazer picking de um único item, chamada pelo picking manual ou por pedido."""
        try:
            item_def = self.get_item(sku)
            loc_picking = self.get_localizacao(id_local_picking)

            plano_picking = self._encontrar_lotes_para_picking(sku, quantidade, estrategia_picking)
            timestamp_mov = datetime.datetime.now()

            for id_local_origem, lotes_a_pegar in plano_picking:
                loc_origem = self.get_localizacao(id_local_origem)
                qtd_local = sum(l.quantidade for l in lotes_a_pegar)

                try:
                    # Remove da origem (usando FIFO interno na transferência)
                    lotes_removidos = loc_origem.remover_item(sku, qtd_local, 'FIFO', self.catalogo_itens)
                    # Adiciona à área de picking (novo lote)
                    id_lote_picking = timestamp_mov.isoformat() + f"_{sku}_P" + (f"_{pedido_id}" if pedido_id else "")
                    lote_adicionado = Lote(qtd_local, timestamp_mov, id_lote_picking)
                    loc_picking.adicionar_lote(item_def, qtd_local, timestamp_mov, self.catalogo_itens)

                    self.registrar_movimentacao("PICKING", sku, qtd_local,
                                               id_local_origem=id_local_origem, id_local_destino=id_local_picking,
                                               detalhes=f"Picking de {id_local_origem} (Estratégia Global: {estrategia_picking})",
                                               lotes_afetados=[lote_adicionado], # O lote que chegou no picking
                                               pedido_id=pedido_id)
                except Exception as e:
                    print(f"ERRO CRÍTICO [Picking Item {sku}]: Falha ao mover de {id_local_origem}: {e}")
                    # Rollback complexo - idealmente marcar item/pedido como falha
                    # Por agora, apenas reporta e potencialmente falha o item/pedido
                    return False # Falha para este item

            print(f"INFO: Picking de {quantidade}x {sku} para {id_local_picking} concluído.")
            return True # Sucesso para este item

        except (ItemNaoEncontradoError, LocalizacaoNaoEncontradaError, EstoqueInsuficienteError,
                CapacidadeExcedidaError, ValueError, WMSError) as e:
            print(f"ERRO [Picking Item {sku}]: {e}")
            return False # Falha para este item

    def fazer_picking(self, sku: str, quantidade: int, estrategia_picking='FIFO', id_local_picking="PICKING_AREA"):
        """Interface pública para picking manual de um item."""
        self._executar_picking_item(sku, quantidade, estrategia_picking, id_local_picking, pedido_id=None)

    def processar_picking_pedido(self, pedido_id: str, id_local_picking="PICKING_AREA", estrategia_picking='FIFO'):
        """Processa o picking para todos os itens de um pedido."""
        try:
            pedido = self.buscar_pedido(pedido_id)
            if pedido.status != StatusPedido.PENDENTE:
                 raise OperacaoInvalidaStatusPedidoError(f"Não é possível iniciar picking do pedido {pedido_id}. Status atual: {pedido.status}")

            print(f"\n--- Iniciando Picking Pedido {pedido_id} ---")
            pedido.mudar_status(StatusPedido.EM_PICKING)

            sucesso_geral = True
            itens_falhados = []

            for sku, quantidade in pedido.itens_solicitados.items():
                 print(f"  -> Tentando picking de {quantidade}x {sku}")
                 sucesso_item = self._executar_picking_item(sku, quantidade, estrategia_picking, id_local_picking, pedido.pedido_id)
                 if not sucesso_item:
                     sucesso_geral = False
                     itens_falhados.append(sku)
                     print(f"  *** Falha no picking do item {sku} para o pedido {pedido_id} ***")
                     # Poderia decidir parar o pedido aqui, ou continuar com os outros itens

            if sucesso_geral:
                 pedido.mudar_status(StatusPedido.PICKING_COMPLETO)
                 print(f"--- Picking Pedido {pedido_id} concluído com sucesso ---")
            else:
                 pedido.mudar_status(StatusPedido.PICKING_FALHOU)
                 print(f"--- Picking Pedido {pedido_id} concluído com FALHAS (itens: {', '.join(itens_falhados)}) ---")

        except (PedidoNaoEncontradoError, OperacaoInvalidaStatusPedidoError) as e:
             print(f"ERRO [Processar Picking Pedido]: {e}")
        except Exception as e:
             print(f"ERRO inesperado [Processar Picking Pedido {pedido_id}]: {e}")
             # Tenta marcar o pedido como falha se possível
             try:
                 pedido = self.buscar_pedido(pedido_id)
                 if pedido.status == StatusPedido.EM_PICKING:
                     pedido.mudar_status(StatusPedido.PICKING_FALHOU)
             except PedidoNaoEncontradoError:
                 pass # Pedido não encontrado nem antes.

    def _executar_expedicao_item(self, sku: str, quantidade: int, id_local_origem: str, strategy: str, pedido_id: str | None) -> bool:
        """Lógica interna para expedir um item."""
        try:
            item_def = self.get_item(sku) # Valida SKU
            loc_origem = self.get_localizacao(id_local_origem)

            lotes_removidos = loc_origem.remover_item(sku, quantidade, strategy, self.catalogo_itens)

            self.registrar_movimentacao("EXPEDICAO", sku, quantidade,
                                       id_local_origem=id_local_origem,
                                       detalhes=f"Mercadoria expedida (Estratégia local: {strategy}).",
                                       lotes_afetados=lotes_removidos,
                                       pedido_id=pedido_id)
            print(f"INFO: Expedido {quantidade}x {sku} de {id_local_origem}.")
            return True

        except (ItemNaoEncontradoError, LocalizacaoNaoEncontradaError, EstoqueInsuficienteError, ValueError) as e:
            print(f"ERRO [Expedir Item {sku}]: {e}")
            return False

    def expedir_mercadoria(self, sku: str, quantidade: int, id_local_origem="PICKING_AREA", strategy='FIFO'):
        """Interface pública para expedição manual."""
        self._executar_expedicao_item(sku, quantidade, id_local_origem, strategy, pedido_id=None)

    def expedir_pedido(self, pedido_id: str, id_local_origem="PICKING_AREA", strategy='FIFO'):
         """Expede todos os itens associados a um pedido a partir da área de origem."""
         try:
             pedido = self.buscar_pedido(pedido_id)
             if pedido.status not in [StatusPedido.PICKING_COMPLETO, StatusPedido.PICKING_FALHOU]:
                  # Permitimos expedir mesmo com falha no picking (parcialmente)
                  raise OperacaoInvalidaStatusPedidoError(f"Não é possível expedir o pedido {pedido_id}. Status atual: {pedido.status} (requer PICKING_COMPLETO ou PICKING_FALHOU).")

             print(f"\n--- Iniciando Expedição Pedido {pedido_id} ---")

             sucesso_geral = True
             itens_falhados = []

             # Idealmente, verificaríamos o que REALMENTE está na área de picking para este pedido.
             # Simplificação: tentamos expedir o que foi originalmente solicitado.
             for sku, quantidade in pedido.itens_solicitados.items():
                  print(f"  -> Tentando expedir {quantidade}x {sku}")
                  sucesso_item = self._executar_expedicao_item(sku, quantidade, id_local_origem, strategy, pedido.pedido_id)
                  if not sucesso_item:
                      sucesso_geral = False
                      itens_falhados.append(sku)
                      print(f"  *** Falha na expedição do item {sku} para o pedido {pedido_id} (pode já ter falhado no picking) ***")
                      # Loga, mas continua tentando expedir outros itens

             # Atualiza status APENAS se todos itens (solicitados) foram expedidos com sucesso
             # Ou seja, se o picking foi completo E a expedição funcionou para todos.
             if pedido.status == StatusPedido.PICKING_COMPLETO and sucesso_geral:
                  pedido.mudar_status(StatusPedido.EXPEDIDO)
                  print(f"--- Expedição Pedido {pedido_id} concluída com sucesso ---")
             else:
                  print(f"--- Expedição Pedido {pedido_id} concluída (Potencialmente PARCIAL devido a falhas no picking ou expedição) ---")
                  # O status permanece PICKING_COMPLETO ou PICKING_FALHOU para indicar que não foi 100%

         except (PedidoNaoEncontradoError, OperacaoInvalidaStatusPedidoError) as e:
              print(f"ERRO [Expedir Pedido]: {e}")
         except Exception as e:
              print(f"ERRO inesperado [Expedir Pedido {pedido_id}]: {e}")

    def consultar_estoque_total(self, sku_item: str) -> int: # Mesmo de antes
        if sku_item not in self.catalogo_itens: return 0
        total = 0
        for local in self.localizacoes.values():
            total += local.verificar_estoque(sku_item)
        return total

    def inventario_geral(self): # Mesmo de antes, sem o hack
        print("\n--- Inventário Geral ---")
        if not self.localizacoes: print("Armazém sem localizações."); return
        for id_local, local_obj in sorted(self.localizacoes.items()):
            # Força o recálculo para exibir a ocupação correta se tivéssemos mantido no __str__
            # local_obj.calcular_ocupacao_atual(self.catalogo_itens, recalcular=True)
            print(local_obj)
        print("------------------------\n")

    def mostrar_log(self): # Adaptado para mostrar lote e pedido_id
        print("\n--- Log de Movimentações ---")
        if not self.log_movimentacoes: print("Nenhuma movimentação."); return
        for entry in self.log_movimentacoes:
            lotes_str = " | ".join([f"Lote({l['id_lote']}, Qtd:{l['quantidade']}, TS:{l['timestamp_entrada']})" for l in entry.get('lotes', [])])
            pedido_str = f" Pedido:[{entry.get('pedido_id', 'N/A')}]" if entry.get('pedido_id') else ""
            print(f"{entry['timestamp']} - {entry['tipo']} - SKU: {entry['sku']}, Qtd: {entry['quantidade']}, "
                  f"Orig: {entry.get('origem','-')}, Dest: {entry.get('destino','-')}{pedido_str} "
                  f"Lotes:[{lotes_str if lotes_str else '-'}] Det: {entry['detalhes']}")
        print("--------------------------\n")

    def mostrar_pedidos(self, status_filtro: StatusPedido | None = None):
         """Mostra os pedidos, opcionalmente filtrando por status."""
         print("\n--- Lista de Pedidos ---")
         if not self.pedidos: print("Nenhum pedido registrado."); return

         pedidos_filtrados = list(self.pedidos.values())
         if status_filtro:
             if isinstance(status_filtro, StatusPedido):
                pedidos_filtrados = [p for p in pedidos_filtrados if p.status == status_filtro]
                print(f"Filtrando por status: {status_filtro}")
             else:
                 print("AVISO: Filtro de status inválido, mostrando todos.")

         if not pedidos_filtrados:
             print("Nenhum pedido encontrado com o filtro especificado.")
             return

         pedidos_filtrados.sort(key=lambda p: p.timestamp_criacao) # Ordena por criação
         for pedido in pedidos_filtrados:
             print(pedido) # Usa o __str__ do Pedido
         print("----------------------\n")

    # --- Métodos de Persistência ---

    def _salvar_estado(self, filename="armazem_estado.json"):
        """Salva o estado atual do armazém em um arquivo JSON."""
        print(f"\nINFO: Salvando estado em {filename}...")
        estado = {
            'nome': self.nome,
            'catalogo_itens': {sku: item.to_dict() for sku, item in self.catalogo_itens.items()},
            'localizacoes': {id_loc: loc.to_dict() for id_loc, loc in self.localizacoes.items()},
            'log_movimentacoes': self.log_movimentacoes, # Já serializado ao adicionar
            'pedidos': {pid: pedido.to_dict() for pid, pedido in self.pedidos.items()}
        }
        try:
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(estado, f, indent=4, ensure_ascii=False)
            print("INFO: Estado salvo com sucesso.")
        except IOError as e:
            print(f"ERRO CRÍTICO: Não foi possível salvar o estado em {filename}. Erro: {e}")
        except TypeError as e:
             print(f"ERRO CRÍTICO: Erro de tipo ao serializar para JSON. Verifique to_dict(). Erro: {e}")

    @classmethod
    def carregar_estado(cls, filename="armazem_estado.json") -> 'ArmazemWMS':
        """Carrega o estado do armazém de um arquivo JSON ou cria um novo."""
        if os.path.exists(filename):
            print(f"INFO: Carregando estado de {filename}...")
            try:
                with open(filename, 'r', encoding='utf-8') as f:
                    estado = json.load(f)

                # Cria instância básica
                armazem = cls(nome=estado.get('nome', 'Armazém Carregado Sem Nome'))

                # Carrega catálogo
                for sku, item_data in estado.get('catalogo_itens', {}).items():
                    try:
                         # Usa ** para passar o dicionário como kwargs para o construtor
                        armazem.catalogo_itens[sku] = Item(**item_data)
                    except (TypeError, ValueError) as e:
                         print(f"AVISO: Erro ao carregar item {sku}: {e}. Item ignorado.")

                # Carrega localizações (precisa do catálogo JÁ carregado)
                for id_loc, loc_data in estado.get('localizacoes', {}).items():
                     try:
                        # Passa o catálogo para o from_dict da localização
                        armazem.localizacoes[id_loc] = Localizacao.from_dict(loc_data, armazem.catalogo_itens)
                     except (TypeError, ValueError, KeyError) as e:
                         print(f"AVISO: Erro ao carregar localização {id_loc}: {e}. Localização ignorada.")

                # Carrega log (deve ser direto se foi bem salvo)
                log_data = estado.get('log_movimentacoes', [])
                # Validar cada entrada seria bom, mas vamos confiar por ora
                armazem.log_movimentacoes = log_data

                 # Carrega pedidos
                for pid, pedido_data in estado.get('pedidos', {}).items():
                    try:
                        pedido = Pedido.from_dict(pedido_data)
                        # Verifica se os itens do pedido AINDA existem no catálogo carregado
                        itens_ok = True
                        for sku in pedido.itens_solicitados:
                             if sku not in armazem.catalogo_itens:
                                 print(f"AVISO: Pedido {pid} contém SKU {sku} que não está mais no catálogo. Pedido não será carregado completamente ativo.")
                                 # Poderia marcar como cancelado ou inconsistente
                                 itens_ok = False
                                 break
                        if itens_ok:
                             armazem.pedidos[pid] = pedido
                        else:
                            # Opção: Carregar como cancelado
                            print(f"INFO: Pedido {pid} carregado como CANCELADO devido a SKUs inexistentes.")
                            pedido.mudar_status(StatusPedido.CANCELADO)
                            armazem.pedidos[pid] = pedido

                    except (TypeError, ValueError, KeyError) as e:
                        print(f"AVISO: Erro ao carregar pedido {pid}: {e}. Pedido ignorado.")

                print(f"INFO: Estado carregado com sucesso. Armazém '{armazem.nome}'.")
                # Recalcular ocupações iniciais ao carregar? Pode ser bom.
                print("INFO: Recalculando ocupação inicial das localizações...")
                for loc in armazem.localizacoes.values():
                     loc.calcular_ocupacao_atual(armazem.catalogo_itens, recalcular=True)

                return armazem

            except (IOError, json.JSONDecodeError, KeyError, TypeError) as e:
                print(f"ERRO: Falha ao carregar estado de {filename}. Erro: {e}.")
                print("INFO: Iniciando um novo armazém vazio.")
                # Retorna um armazém novo/vazio com um nome padrão
                return cls(nome="Armazém Padrão (Novo)")
        else:
            print(f"INFO: Arquivo {filename} não encontrado. Iniciando um novo armazém vazio.")
            return cls(nome="Armazém Padrão (Novo)")

In [ ]:
# --- Exemplo de Uso ---

if __name__ == "__main__":
    # 1. Criar o Armazém
    meu_armazem = ArmazemWMS("Armazém Avançado SP")
    print(f"Armazém '{meu_armazem.nome}' criado.")

    # 2. Adicionar Itens com Volume/Peso ao Catálogo
    # Volumes pequenos para caberem mais
    item1 = Item("SKU001", "Parafuso M6", volume=0.01, peso=0.05)
    item2 = Item("SKU002", "Porca M6", volume=0.005, peso=0.02)
    item3 = Item("SKU003", "Arruela M6", volume=0.002, peso=0.01)
    meu_armazem.adicionar_item_catalogo(item1)
    meu_armazem.adicionar_item_catalogo(item2)
    meu_armazem.adicionar_item_catalogo(item3)

    # 3. Adicionar Localizações com Capacidade ao Armazém
    # Capacidade de Volume: ex: 1.0 m³ ou equivalente
    # Capacidade de Peso: ex: 50 kg
    # Capacidade de Quantidade: ex: 1000 itens
    loc_recv = Localizacao("RECEBIMENTO", capacidade_maxima=10.0, tipo_capacidade='volume') # Grande volume para recebimento
    loc_a101 = Localizacao("A1-01", capacidade_maxima=1.0, tipo_capacidade='volume')
    loc_a102 = Localizacao("A1-02", capacidade_maxima=0.5, tipo_capacidade='volume') # Menor
    loc_b101 = Localizacao("B1-01", capacidade_maxima=50, tipo_capacidade='peso')   # Baseado em peso
    loc_pick = Localizacao("PICKING_AREA", capacidade_maxima=2.0, tipo_capacidade='volume')
    loc_exp = Localizacao("EXPEDICAO", capacidade_maxima=float('inf')) # Sem limite prático

    meu_armazem.adicionar_localizacao(loc_recv)
    meu_armazem.adicionar_localizacao(loc_a101)
    meu_armazem.adicionar_localizacao(loc_a102)
    meu_armazem.adicionar_localizacao(loc_b101)
    meu_armazem.adicionar_localizacao(loc_pick)
    meu_armazem.adicionar_localizacao(loc_exp)

    meu_armazem.inventario_geral()

    # 4. Simular Operações com Tratamento de Erros e Estratégias
    print("\n--- Simulando Operações WMS Avançadas ---")

    # Recebimento (Deve funcionar)
    print("\n>>> Recebendo mercadorias <<<")
    meu_armazem.receber_mercadoria("SKU001", 100) # 100 * 0.01 = 1.0 de volume
    meu_armazem.receber_mercadoria("SKU002", 150) # 150 * 0.005 = 0.75 de volume
    # Recebimento com erro (SKU inexistente)
    meu_armazem.receber_mercadoria("SKU999", 10)
    meu_armazem.inventario_geral()

    # Guardar (Armazenagem)
    print("\n>>> Guardando mercadorias <<<")
    # Tentar guardar SKU001 (100 und = 1.0 vol) - Deve ir para A1-01 (cap 1.0)
    meu_armazem.guardar_mercadoria("SKU001", 90, "RECEBIMENTO", estrategia_guarda='EXISTENTE_SKU') # 90 * 0.01 = 0.9 vol
    # Tentar guardar SKU002 (150 und = 0.75 vol) - Deve ir para A1-02 (cap 0.5) - NÃO CABE! Deveria escolher outra ou dar erro?
    # Ah, _encontrar_local só pega locais com capacidade. Deve ir para B1-01 (baseada em peso, mas aqui volume baixo) ou outra
    meu_armazem.guardar_mercadoria("SKU002", 150, "RECEBIMENTO", estrategia_guarda='MELHOR_FIT') # 150 * 0.005 = 0.75 vol -> Cabe em A1-01 ainda? Ou B1-01?
    # Tentar guardar o restante de SKU001 (10 und = 0.1 vol) - A1-01 deve ter 0.1 vol livre. Deve ir pra lá.
    meu_armazem.guardar_mercadoria("SKU001", 10, "RECEBIMENTO", estrategia_guarda='EXISTENTE_SKU')
    meu_armazem.inventario_geral()

    # Testar Capacidade
    print("\n>>> Testando Capacidade <<<")
    # Tentar adicionar mais SKU001 em A1-01 (que está cheio 1.0 vol)
    meu_armazem.receber_mercadoria("SKU001", 5, "RECEBIMENTO") # Adiciona mais no recebimento
    meu_armazem.guardar_mercadoria("SKU001", 5, "RECEBIMENTO", estrategia_guarda='EXISTENTE_SKU') # Deve falhar ou escolher outro local
    meu_armazem.inventario_geral()

    # Simular outra entrada para teste FIFO/LIFO
    print("\n>>> Recebendo mais tarde <<<")
    import time
    time.sleep(1) # Pequena pausa para garantir timestamp diferente
    meu_armazem.receber_mercadoria("SKU001", 50, "RECEBIMENTO") # Novo lote de SKU001
    meu_armazem.guardar_mercadoria("SKU001", 50, "RECEBIMENTO", estrategia_guarda='MELHOR_FIT') # Onde vai? A1-01 cheio, A1-02? B1-01?

    meu_armazem.inventario_geral()

    # Picking (FIFO)
    print("\n>>> Fazendo Picking FIFO <<<")
    # Temos SKU001: Lote antigo (100 unds total em A1-01?) + Lote novo (50 unds em B1-01?). Total 150.
    # Pedir 120 unidades FIFO. Deve pegar as 100 do lote antigo (A1-01) e 20 do lote novo (B1-01?)
    meu_armazem.fazer_picking("SKU001", 120, estrategia_picking='FIFO', id_local_picking="PICKING_AREA")
    meu_armazem.inventario_geral() # Verificar estoque em A1-01 e onde o segundo lote estava

    # Picking (LIFO)
    print("\n>>> Fazendo Picking LIFO <<<")
    # Temos SKU002 (150 unidades, digamos que foram para B1-01). E sobraram 30 SKU001 no lote mais novo.
    meu_armazem.receber_mercadoria("SKU002", 20, "RECEBIMENTO") # Mais 20 de SKU002 (novo lote)
    time.sleep(1)
    meu_armazem.guardar_mercadoria("SKU002", 20, "RECEBIMENTO") # Guarda o novo lote
    meu_armazem.inventario_geral()
    # Pedir 40 LIFO. Deve pegar as 20 do lote mais novo, e 20 do lote anterior.
    meu_armazem.fazer_picking("SKU002", 40, estrategia_picking='LIFO', id_local_picking="PICKING_AREA")
    meu_armazem.inventario_geral() # Verificar estoque de SKU002

    # Testar Picking com Estoque Insuficiente
    print("\n>>> Testando Picking Insuficiente <<<")
    meu_armazem.fazer_picking("SKU001", 1000, estrategia_picking='FIFO') # Tenta pegar mais do que existe

    # Expedição
    print("\n>>> Expedindo Mercadorias <<<")
    # Expedir o que foi para PICKING_AREA
    # Tínhamos pego 120 de SKU001 e 40 de SKU002
    meu_armazem.expedir_mercadoria("SKU001", 120, strategy='FIFO', id_local_origem="PICKING_AREA")
    meu_armazem.expedir_mercadoria("SKU002", 40, strategy='FIFO', id_local_origem="PICKING_AREA")
    # Tentar expedir algo que não está lá
    meu_armazem.expedir_mercadoria("SKU003", 10, "PICKING_AREA")
    meu_armazem.inventario_geral() # Área de picking deve estar vazia agora

    # Consultar estoque final
    print("\n>>> Estoque Final <<<")
    print(f"Estoque total de SKU001: {meu_armazem.consultar_estoque_total('SKU001')}") # Deveria ser 30
    print(f"Estoque total de SKU002: {meu_armazem.consultar_estoque_total('SKU002')}") # Deveria ser 130 (150 - 20 + 20 - 40) = 110 ? Revisar calculo! 150 + 20 = 170 total. Tirou 40 LIFO. 170-40 = 130. Correto.
    print(f"Estoque total de SKU003: {meu_armazem.consultar_estoque_total('SKU003')}") # Deve ser 0

    # Mostrar Log Completo
    meu_armazem.mostrar_log()

In [9]:
#  Exemplo de Uso Atualizado com Pedidos e Persistência

if __name__ == "__main__":
    ARQUIVO_ESTADO = "meu_armazem_wms.json"
    # --- Carregar ou Criar Armazém ---
    meu_armazem = ArmazemWMS.carregar_estado(ARQUIVO_ESTADO)

    # Tenta salvar o estado no final, mesmo se ocorrer um erro durante a execução
    try:
        # --- Configuração Inicial (só executa se for novo armazém) ---
        if not meu_armazem.catalogo_itens: # Se o catálogo está vazio, provavelmente é novo
             print("\n--- Configurando Armazém Inicial ---")
             # Adicionar Itens
             item1 = Item("SKU001", "Parafuso M6", volume=0.01, peso=0.05)
             item2 = Item("SKU002", "Porca M6", volume=0.005, peso=0.02)
             item3 = Item("SKU003", "Arruela M6", volume=0.002, peso=0.01)
             meu_armazem.adicionar_item_catalogo(item1)
             meu_armazem.adicionar_item_catalogo(item2)
             meu_armazem.adicionar_item_catalogo(item3)

             # Adicionar Localizações
             loc_recv = Localizacao("RECEBIMENTO", capacidade_maxima=10.0, tipo_capacidade='volume')
             loc_a101 = Localizacao("A1-01", capacidade_maxima=1.0, tipo_capacidade='volume')
             loc_a102 = Localizacao("A1-02", capacidade_maxima=0.5, tipo_capacidade='volume')
             loc_b101 = Localizacao("B1-01", capacidade_maxima=50, tipo_capacidade='peso')
             loc_pick = Localizacao("PICKING_AREA", capacidade_maxima=2.0, tipo_capacidade='volume')
             loc_exp = Localizacao("EXPEDICAO", capacidade_maxima=float('inf'))

             meu_armazem.adicionar_localizacao(loc_recv)
             meu_armazem.adicionar_localizacao(loc_a101)
             meu_armazem.adicionar_localizacao(loc_a102)
             meu_armazem.adicionar_localizacao(loc_b101)
             meu_armazem.adicionar_localizacao(loc_pick)
             meu_armazem.adicionar_localizacao(loc_exp)

             # Carga inicial (Exemplo)
             print("\n--- Carga Inicial Exemplo ---")
             meu_armazem.receber_mercadoria("SKU001", 200)
             meu_armazem.receber_mercadoria("SKU002", 300)
             meu_armazem.guardar_mercadoria("SKU001", 180, "RECEBIMENTO") # Tenta guardar 1.8 vol -> Nao cabe em A1 nem A2. Vai p/ B1?
             meu_armazem.guardar_mercadoria("SKU002", 250, "RECEBIMENTO") # Tenta guardar 1.25 vol
             meu_armazem.inventario_geral()


        # --- Operações Normais ---
        meu_armazem.inventario_geral()
        meu_armazem.mostrar_pedidos() # Mostra pedidos existentes

        # Criar Novos Pedidos (Exemplo)
        print("\n--- Criando Novos Pedidos ---")
        try:
             # Pedido que deve funcionar (se houver estoque)
             itens_p1 = {"SKU001": 30, "SKU002": 50}
             pedido1 = Pedido("Cliente Feliz", itens_p1)
             meu_armazem.adicionar_pedido(pedido1)

             # Pedido com item que pode faltar
             itens_p2 = {"SKU001": 500, "SKU003": 10} # SKU003 não tem estoque, SKU001 pode faltar
             pedido2 = Pedido("Cliente Teste", itens_p2)
             meu_armazem.adicionar_pedido(pedido2)

             # Pedido com SKU inválido (não deve ser adicionado)
             itens_p3 = {"SKU999": 5}
             pedido3 = Pedido("Cliente Erro", itens_p3)
             meu_armazem.adicionar_pedido(pedido3) # Deve falhar na adição

        except (ValueError, ItemNaoEncontradoError) as e:
             print(f"ERRO ao criar/adicionar pedido: {e}")

        meu_armazem.mostrar_pedidos(StatusPedido.PENDENTE)

        # Processar Picking dos Pedidos Pendentes
        print("\n--- Processando Picking de Pedidos Pendentes ---")
        pedidos_pendentes = [p for p in meu_armazem.pedidos.values() if p.status == StatusPedido.PENDENTE]
        for pedido in pedidos_pendentes:
            meu_armazem.processar_picking_pedido(pedido.pedido_id, estrategia_picking='FIFO')

        meu_armazem.inventario_geral()
        meu_armazem.mostrar_pedidos() # Mostrar status atualizado

        # Expedir Pedidos com Picking Completo
        print("\n--- Expedindo Pedidos com Picking Completo ---")
        pedidos_prontos = [p for p in meu_armazem.pedidos.values() if p.status == StatusPedido.PICKING_COMPLETO]
        for pedido in pedidos_prontos:
             meu_armazem.expedir_pedido(pedido.pedido_id)

        meu_armazem.inventario_geral() # Verificar se área de picking foi limpa
        meu_armazem.mostrar_pedidos() # Mostrar status final

        # Mostrar log completo da sessão
        meu_armazem.mostrar_log()

    except Exception as e_main:
        # Captura qualquer erro inesperado durante a simulação
        print(f"\n!!!! ERRO INESPERADO DURANTE A EXECUÇÃO !!!!")
        import traceback
        traceback.print_exc() # Imprime o stack trace do erro

    finally:
        # --- Salvar Estado Final ---
        meu_armazem._salvar_estado(ARQUIVO_ESTADO)

INFO: Arquivo meu_armazem_wms.json não encontrado. Iniciando um novo armazém vazio.

--- Configurando Armazém Inicial ---
INFO: Item SKU001 (Parafuso M6) adicionado/atualizado no catálogo.
INFO: Item SKU002 (Porca M6) adicionado/atualizado no catálogo.
INFO: Item SKU003 (Arruela M6) adicionado/atualizado no catálogo.
INFO: Localização RECEBIMENTO adicionada.
INFO: Localização A1-01 adicionada.
INFO: Localização A1-02 adicionada.
INFO: Localização B1-01 adicionada.
INFO: Localização PICKING_AREA adicionada.
INFO: Localização EXPEDICAO adicionada.

--- Carga Inicial Exemplo ---
ERRO CRÍTICO [Guardar - Adição Destino]: name 'lotes_removidos_origem' is not defined. Iniciando rollback.
ERRO CRÍTICO IMPOSSÍVEL DE REVERTER [Guardar - Rollback]: name 'lotes_removidos_origem' is not defined. Estado inconsistente!
ERRO CRÍTICO [Guardar - Adição Destino]: name 'lotes_removidos_origem' is not defined. Iniciando rollback.
ERRO CRÍTICO IMPOSSÍVEL DE REVERTER [Guardar - Rollback]: name 'lotes_removid